In [3]:
import httpx
import asyncio
from tenacity import retry, stop_after_attempt, wait_fixed

base_url = "http://localhost:8000"

@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
async def add_document_test():
    async with httpx.AsyncClient(timeout=60.0) as client:
        document_data = {
            "content": "This is a sample document about machine learning.",
            "metadata": {"source": "ML Course"}
        }
        response = await client.post(f"{base_url}/add_document", json=document_data)
        return response.json()

async def test_chatbot():
    print("Adding Document...")
    add_doc_response = await add_document_test()
    print("Response from /add_document:", add_doc_response)

# Run the test
await test_chatbot()


Adding Document...
Response from /add_document: {'message': 'Document added successfully'}


In [21]:
async def test_chatbot():
    print("Starting Chat...")
    
    conversation_id = "test-conversation"
    
    # First, let's add some test document
    test_doc = {
        "content": "In the year 2042, humanity had conquered space. Lunar colonies were thriving, and Mars was being terraformed. But one day, the moon vanished. Scientists were baffled. No trace of it was found. Theories ranged from alien abduction to a natural anomaly.You, a young astronomer, were tasked with investigating. Armed with cutting-edge technology, you embarked on a perilous journey to the moon's former orbit. There, you discovered a strange energy field. As you probed deeper, you realized it was a portal to another dimension. Intrigued, you ventured through.On the other side, you found a parallel Earth, eerily similar to your own. But there was one crucial difference: the moon was intact, hanging in the sky. You realized that an unknown force had been siphoning energy from the moon, creating the portal.Your mission became clear: to find a way to return the moon to its rightful place. But as you worked, you discovered a dark secret about the beings behind the energy drain. They were not benevolent.",
        "metadata": {"source": "randomstory.txt"}
        }
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        # Add the document
        await client.post(f"{base_url}/add_document", json=test_doc)
        
        # Initial message
        initial_message = {
            "query": "Hello! What can you do?",
            "conversation_id": conversation_id
        }
        chat_response = await client.post(f"{base_url}/chat", json=initial_message)
        print("Response from chatbot:", chat_response.json())
        
        # Follow-up question
        follow_up_message = {
            "query": "What do you think caused the moon to vanish in the first place?",
            "conversation_id": conversation_id
        }
        follow_up_response = await client.post(f"{base_url}/chat", json=follow_up_message)
        print("Follow-up response:", follow_up_response.json())
        
        # Delete the conversation
        delete_response  = await client.delete(f"{base_url}/conversations/{conversation_id}")
        print("Delete Response:", delete_response.json())

# Running the test using asyncio
await test_chatbot()

Starting Chat...
Response from chatbot: {'answer': 'I can answer questions, have conversations, generate text (like stories or articles), and more. How may I assist you today?', 'conversation_id': 'test-conversation', 'sources': []}
Follow-up response: {'answer': 'I don\'t actually have the capability to think about such a hypothetical scenario or provide an opinion on it because that would require creative writing, which is beyond my programming. However, if we were to speculate hypothetically:\n\nOne common explanation for something "vanishing" might involve some kind of natural disaster, alien intervention, or even a sci-fi plot where the moon has been concealed by technology or magic.\n\nIt\'s important to note that this is purely speculative and not based on scientific fact.', 'conversation_id': 'test-conversation', 'sources': []}
Delete Response: {'detail': "'Chroma' object has no attribute '_persist_client'"}


In [7]:
async def test_document_reader():
    print("Starting document upload and read test...")

    # Define the file path for the document to be uploaded
    file_path = "samples/sample.pptx"  # Replace with the path to your test document
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        # Open the file in binary mode for upload
        with open(file_path, "rb") as f:
            files = {'file': (file_path, f, 'multipart/form-data')}
            
            # Upload the document to the API
            upload_response = await client.post(f"{base_url}/read_document", files=files)
            print("Response from document upload:", upload_response.json())
await test_document_reader()

Starting document upload and read test...
Response from document upload: {'detail': 'Unsupported file extension: '}


In [15]:
# test_document_reader.py
import os
import asyncio
import httpx

base_url = "http://127.0.0.1:8000"

async def test_document_reader():
    print("Starting Document Reader Test...")

    # Define the path to a test document
    test_file_path = "samples/sample.pptx"

    # Ensure the test document exists
    if not os.path.exists(test_file_path):
        raise FileNotFoundError(f"The test file {test_file_path} does not exist.")

    async with httpx.AsyncClient(timeout=60.0) as client:
        # Upload the document and extract text
        with open(test_file_path, "rb") as file:
            files = {"file": (os.path.basename(test_file_path), file)}
            response = await client.post(f"{base_url}/extract-text/", files=files)
        
        print("Response from document reader:", response.json())

await test_document_reader()

Starting Document Reader Test...
Response from document reader: {'text': 'Various Image Segmentation Techniques:\nDifferent types of Image Segmentation Techniques\nThresholding technique segmentation\nHistogram based segmentation\nRegion based segmentation\nEdge based segmentation\nClustering based segmentation\nMorphological Transforms and\nTexture based segmentation approaches\nThresholding technique segmentation\nSegmentation algorithms based on thresholding approach are suitable for images where there is distinct difference between object and background.\nMain Goal: divide an image into two distinct regions (object and background) directly based on intensity values and their properties\nTypes: Global, Variable, Multiple\n\nOriginal coins image\n2)Histogram based segmentation\nHistogram of an image is a plot between intensity levels. \nDeep valleys are used to separate different peaks of histogram. \nHistogram peaks are tall, narrow, symmetric.\n3) Region based Segmentation\nThe reg

In [6]:
import requests

url = "http://127.0.0.1:8000/conversations/12345"
payload = {
    "conversation_id": "12345",
    "query": "Where is it located?"
}

response = requests.delete(url)
print(response.json())


{'message': 'Conversation 12345 deleted successfully'}


In [2]:
def format_chat_history(chat_history):
    formatted_history = []
    for message in chat_history:
        if isinstance(message, HumanMessage):
            formatted_history.append(f"<|im_start|>user\n{message.content}<|im_end|>")
        elif isinstance(message, AIMessage):
            formatted_history.append(f"<|im_start|>assistant\n{message.content}<|im_end|>")
    return "\n".join(formatted_history)

def get_response_from_chain(chain, query):
    try:
        # Format the query if needed
        formatted_query = query
        if not query.startswith("<|im_start|>"):
            formatted_query = f"<|im_start|>user\n{query}<|im_end|>"
            
        response = chain.invoke({"question": formatted_query})
        answer = response.get("answer", "I couldn't find an answer.")
        
        # Ensure the answer has the proper tokens
        if not answer.startswith("<|im_start|>"):
            answer = f"<|im_start|>assistant\n{answer}<|im_end|>"
            
        source_documents = response.get("source_documents", [])
        return answer, source_documents
    except Exception as e:
        print(f"Error in get_response_from_chain: {str(e)}")
        return "<|im_start|>assistant\nI encountered an error while processing your request.<|im_end|>", []


{'answer': 'What are some popular tourist attractions in Paris, France?', 'conversation_id': '12345', 'sources': []}


In [19]:
import asyncio
import httpx

async def test_quiz_generation():
    print("Testing Quiz Generation...")
    
    # Simple test document
    test_doc = {
        "content": "Python is a popular programming language. It was created by Guido van Rossum in 1991. Python is known for its simple syntax and readability. Python supports multiple programming paradigms including procedural, object-oriented, and functional programming.",
        "metadata": {
            "source": "test.txt",
            "conversation_id": "test-convo"
        }
    }
    
    async with httpx.AsyncClient(timeout=30.0) as client:
        try:
            # Generate quiz
            response = await client.post("http://localhost:8000/generate_quiz", json=test_doc)
            print("\nAPI Response Status:", response.status_code)
            
            if response.status_code == 200:
                quiz_data = response.json()
                print("\nGenerated Questions:")
                for i, q in enumerate(quiz_data["quiz"], 1):
                    print(f"\nQ{i}: {q['question']}")
                    print("Options:", q['options'])
                    print("Answer:", q['correct_option'])
            else:
                print("Error response:", response.text)
        except Exception as e:
            print(f"Error during test: {str(e)}")

await test_quiz_generation()

Testing Quiz Generation...

API Response Status: 200

Generated Questions:

Q1: What is Python?
Options: ['A programming language', 'An operating system', 'A web browser', 'A database']
Answer: A programming language

Q2: Who created Python?
Options: ['Guido van Rossum', 'Tim Berners-Lee', 'Linus Torvalds', 'Bill Gates']
Answer: Guido van Rossum

Q3: What does Python support?
Options: ['Procedural programming only', 'Object-oriented programming only', 'Functional programming only', 'Multiple paradigms including procedural, object-oriented, and functional programming']
Answer: Multiple paradigms including procedural, object-oriented, and functional programming

Q4: When was Python created?
Options: ['1980', '1991', '2000', '2010']
Answer: 1991

Q5: What does the name 'Python' mean in English?
Options: ['It means snake in Greek', 'It means nothing in English', 'It means monkey in Spanish', 'It is named after a British actor']
Answer: It means snake in Greek


In [20]:
import asyncio
import httpx
import json

async def test_summarization():
    print("Testing Summarization...")
    
    # Define base URL
    base_url = "http://localhost:8000"
    
    test_doc = {
        "content": """
        Artificial Intelligence (AI) is revolutionizing various industries and aspects of daily life. 
        Machine Learning, a subset of AI, enables systems to learn and improve from experience without being explicitly programmed. 
        Deep Learning, a more advanced form of Machine Learning, uses neural networks with many layers to analyze complex patterns in data. 
        Natural Language Processing (NLP) allows machines to understand, interpret, and generate human language, powering applications like chatbots and language translation services. 
        Computer Vision enables machines to gain high-level understanding from digital images or videos, used in facial recognition and autonomous vehicles. 
        AI ethics is an important consideration, addressing issues like bias in AI systems, privacy concerns, and the societal impact of AI technologies. 
        The future of AI promises further advancements in areas such as healthcare, finance, education, and scientific research, potentially leading to significant improvements in efficiency and quality of life.
        """,
        "metadata": {
            "source": "ai_overview.txt"
        }
    }
    
    print(f"Connecting to: {base_url}/summarize")
    
    async with httpx.AsyncClient(timeout=120.0) as client:
        try:
            # Test server connection first
            health_check = await client.get(f"{base_url}/")
            print(f"Server connection status: {health_check.status_code}")
            
            # Generate summary
            response = await client.post(f"{base_url}/summarize", json=test_doc)
            print("\nAPI Response Status:", response.status_code)
            
            if response.status_code == 200:
                summary_data = response.json()
                
                print("\nGenerated Summary:")
                if "summary" in summary_data:
                    print(summary_data["summary"])
                else:
                    print("Unexpected response format:", json.dumps(summary_data, indent=2))
            else:
                print("Error response:", response.text)
                
        except httpx.ConnectError:
            print(f"Could not connect to server at {base_url}")
            print("Please make sure the FastAPI server is running on port 8000")
        except Exception as e:
            print(f"Error during test: {str(e)}")
            raise

await test_summarization()

Testing Summarization...
Connecting to: http://localhost:8000/summarize
Server connection status: 404

API Response Status: 200

Generated Summary:
The content describes how Artificial Intelligence (AI) is transforming multiple sectors. Key points include:

1. AI's broad impact across industries and daily life.
2. Machine Learning as an enabling technology, capable of learning from data without explicit programming.
3. Deep Learning, a specialized form of ML using neural networks for complex pattern analysis.
4. Natural Language Processing (NLP) allowing machines to understand human language for applications like chatbots and translation.
5. Computer Vision enabling high-level understanding from visual inputs in fields like facial recognition and autonomous driving.
6. The importance of AI ethics in addressing issues related to bias, privacy, and societal impact.
7. Anticipated future advancements in healthcare, finance, education, and research, promising improved efficiency and qualit